<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/fundamental_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import datetime

In [2]:
today = datetime.date.today().strftime('%Y%m%d')

In [3]:
data = {
    'mktId': 'KSQ',
    'trdDd': today,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901',
}
gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_key = requests.post(gen_url, data=data)

In [4]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
r = requests.post(down_url, data={'code':gen_key.text})
r.encoding = 'EUC-KR'

In [5]:
from io import StringIO

df = pd.read_csv(StringIO(r.text))

In [6]:
df

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,3650,-220,-5.68,168891022450
1,054620,APS홀딩스,KOSDAQ,금융,14950,-350,-2.29,304893603950
2,265520,AP시스템,KOSDAQ,반도체,23150,-750,-3.14,335217255050
3,211270,AP위성,KOSDAQ,통신장비,15500,-1150,-6.91,233775712000
4,013720,CBI,KOSDAQ,운송장비·부품,3590,-260,-6.75,182005012250
...,...,...,...,...,...,...,...,...
1509,024060,흥구석유,KOSDAQ,유통,7650,-360,-4.49,114750000000
1510,010240,흥국,KOSDAQ,기계·장비,7520,-290,-3.71,92666673920
1511,189980,흥국에프엔비,KOSDAQ,음식료·담배,2730,-170,-5.86,104796856710
1512,037440,희림,KOSDAQ,기타서비스,5390,-490,-8.33,75042140250


In [7]:
boundary = df.loc[df['시가총액'].between(30000000000,70000000000)]
boundary

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
10,056000,COWON,KOSDAQ,디지털컨텐츠,2860,0,0.00,56399843500
11,065770,CS,KOSDAQ,통신장비,3450,-350,-9.21,66968398500
12,083660,CSA 코스믹,KOSDAQ,화학,1360,-15,-1.09,51590395040
20,050120,ES큐브,KOSDAQ,유통,4090,-110,-2.62,55477111740
22,900290,GRT,KOSDAQ,금융,951,0,0.00,64073625000
...,...,...,...,...,...,...,...,...
1490,039610,화성밸브,KOSDAQ,기계·장비,5910,-340,-5.44,61525464000
1493,097870,효성오앤비,KOSDAQ,화학,8160,-710,-8.00,69278400000
1495,290270,휴네시온,KOSDAQ,소프트웨어,5330,-120,-2.20,51208891760
1496,353190,휴럼,KOSDAQ,음식료·담배,1870,-40,-2.09,68143585400


In [8]:
!pip install OpenDartReader

In [9]:
import OpenDartReader

api_key = '4b0512c473e52e1cf59f8ce8380b2b30904f381a'
dart = OpenDartReader(api_key)

In [10]:
search_start = '2021-09-01'
search_end = '2021-10-06'

#search_date = datetime.date.today().strftime('%Y-%m-%d')

In [11]:
dart_df = dart.list(start=search_start, end=search_end, kind='A')
found = list(set(dart_df['stock_code']).intersection(set(boundary['종목코드'])))
out_df = dart_df[dart_df['stock_code'].isin(found)]

In [12]:
def get_recent_finstate(corp_code='', year=2021):
  quarters = ['11011', '11014', '11012', '11013']
  for q in quarters:
    # '11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서
    df = dart.finstate(corp_code, year, reprt_code=q)
    if type(df) != type(None):
      return df[df['fs_nm'] == '연결재무제표']

In [13]:
def get_incomestatement(df):
  is_df = df.loc[df['sj_nm'] == '손익계산서']
  try:
    t = is_df[['rcept_no', 'thstrm_dt','account_nm', 'thstrm_amount', 'frmtrm_amount', 'thstrm_add_amount', 'frmtrm_add_amount']]
    t.columns = ['published_date', 'date', 'name', 'current03', 'previous03', 'current12', 'previous12']
  except:
    t = is_df[['rcept_no', 'thstrm_dt','account_nm', 'thstrm_amount', 'frmtrm_amount']]
    t.columns = ['published_date', 'date', 'name', 'current03', 'previous03']
  return t

def get_is_score(is_df):
  is_score = {}
  for n in is_df['name']:
    current = str_int(is_df.loc[is_df['name'] == n, 'current03'].values[0])
    previous = str_int(is_df.loc[is_df['name'] == n, 'previous03'].values[0])
    is_score[n] = get_growth_rate(previous, current)
  return is_score

In [14]:
def get_balancesheet(df):
  bs_df = df.loc[df['sj_nm'] == '재무상태표']
  t = bs_df[['rcept_no', 'thstrm_dt','account_nm', 'thstrm_amount', 'frmtrm_amount']]
  t.columns = ['published_date', 'date', 'name', 'current03', 'previous03']
  return t

def get_bs_score(bs_df):
  bs_score = {}
  for n in bs_df['name']:
    current = str_int(bs_df.loc[bs_df['name'] == n, 'current03'].values[0])
    previous = str_int(bs_df.loc[bs_df['name'] == n, 'previous03'].values[0])
    bs_score[n] = get_growth_rate(previous, current)
  return bs_score

In [15]:
def get_growth_rate(previous, current):
  return (current-previous)/abs(previous)

def str_int(s):
  return int(s.replace(',',''))

In [16]:
score_dic = {}

for code in out_df['corp_code']:
  try:
    df = get_recent_finstate(code)
    is_df = get_incomestatement(df)
    is_score = get_is_score(is_df)

    bs_df = get_balancesheet(df)
    bs_score = get_bs_score(bs_df)

    date = out_df.loc[out_df['corp_code'] == code, 'rcept_dt'].values[0]
    name = out_df.loc[out_df['corp_code'] == code, 'corp_name'].values[0]
    print(date)
    print(name)
    print(is_score)
    print(bs_score)
    print()

    t = is_score
    t.update(bs_score)
    score_dic[name] = t
  except:
    print('ERROR', name)
    print()

20210929
나노브릭
{}
{}

20210928
서연탑메탈
{'매출액': 0.18272865589282206, '영업이익': -0.6213504761218102, '법인세차감전 순이익': -0.5883038356162525, '당기순이익': -0.20819625754166612}
{'유동자산': -0.03085613837396068, '비유동자산': -0.008374439030530728, '자산총계': -0.02038905236376283, '유동부채': -0.11658358630875082, '비유동부채': -0.034738146149367946, '부채총계': -0.09941050213174926, '자본금': 0.0, '이익잉여금': 0.07830936951798947, '자본총계': 0.05540376818977009}

20210917
이큐셀
{'매출액': 0.5946777575625294, '영업이익': 0.7633451325396896, '법인세차감전 순이익': 0.9813018644484682, '당기순이익': 0.9721316919875065}
{'유동자산': -0.2370703695783634, '비유동자산': -0.24581652782162192, '자산총계': -0.24148035427064782, '유동부채': -0.7701239556796861, '비유동부채': 1.41164381896257, '부채총계': -0.502050240114592, '자본금': -0.5427179216068487, '이익잉여금': -0.006226826540301651, '자본총계': 3.0341337394145134}

20210917
스카이이앤엠
{'매출액': -0.14140310196929798, '영업이익': 0.19975064286302283, '법인세차감전 순이익': -0.02171366431663808, '당기순이익': -0.025982011078868078}
{'유동자산': 0.281189948339449, '비유동자산': -0.2704

In [18]:
score_df = pd.DataFrame(score_dic).T
score_df

,매출액,영업이익,법인세차감전 순이익,당기순이익,유동자산,비유동자산,자산총계,유동부채,비유동부채,부채총계,자본금,이익잉여금,자본총계
나노브릭,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
서연탑메탈,0.182729,-0.621350,-0.588304,-0.208196,-0.030856,-0.008374,-0.020389,-0.116584,-0.034738,-0.099411,0.000000,0.078309,0.055404
이큐셀,0.594678,0.763345,0.981302,0.972132,-0.237070,-0.245817,-0.241480,-0.770124,1.411644,-0.502050,-0.542718,-0.006227,3.034134
스카이이앤엠,-0.141403,0.199751,-0.021714,-0.025982,0.281190,-0.270409,-0.103651,-0.433025,-0.075614,-0.422837,0.492820,-0.330465,0.713442
포시에스,0.092411,-0.011266,-0.204175,-0.169891,0.157506,0.079073,0.108244,0.080759,0.074166,0.080049,0.048258,0.097788,0.109553
아세아텍,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
효성오앤비,0.029008,-0.221225,0.551540,0.547930,0.110329,0.253659,0.175417,0.137870,-0.170911,0.090120,0.000000,0.031093,0.189630
패션플랫폼,0.539893,27.566627,90.223817,114.768439,0.340536,0.130448,0.276887,0.168593,-0.168017,0.125325,0.000000,0.572256,0.324714
에스엘바이오닉스,0.097334,-18.374621,-0.586949,-0.578525,0.639343,-0.056904,0.133448,0.502452,-0.276396,0.455392,0.038316,-0.110476,0.011180
디엔에이링크,0.183512,-0.216752,0.284567,0.284567,-0.199590,0.795673,-0.102052,-0.153583,0.354319,-0.137116,0.000000,-0.055307,-0.087963


In [19]:
score_df.sort_values(by=['영업이익', '매출액'], ascending=False).head(10)

,매출액,영업이익,법인세차감전 순이익,당기순이익,유동자산,비유동자산,자산총계,유동부채,비유동부채,부채총계,자본금,이익잉여금,자본총계
패션플랫폼,0.539893,27.566627,90.223817,114.768439,0.340536,0.130448,0.276887,0.168593,-0.168017,0.125325,0.000000,0.572256,0.324714
이큐셀,0.594678,0.763345,0.981302,0.972132,-0.237070,-0.245817,-0.241480,-0.770124,1.411644,-0.502050,-0.542718,-0.006227,3.034134
스카이이앤엠,-0.141403,0.199751,-0.021714,-0.025982,0.281190,-0.270409,-0.103651,-0.433025,-0.075614,-0.422837,0.492820,-0.330465,0.713442
포시에스,0.092411,-0.011266,-0.204175,-0.169891,0.157506,0.079073,0.108244,0.080759,0.074166,0.080049,0.048258,0.097788,0.109553
디엔에이링크,0.183512,-0.216752,0.284567,0.284567,-0.199590,0.795673,-0.102052,-0.153583,0.354319,-0.137116,0.000000,-0.055307,-0.087963
효성오앤비,0.029008,-0.221225,0.551540,0.547930,0.110329,0.253659,0.175417,0.137870,-0.170911,0.090120,0.000000,0.031093,0.189630
서연탑메탈,0.182729,-0.621350,-0.588304,-0.208196,-0.030856,-0.008374,-0.020389,-0.116584,-0.034738,-0.099411,0.000000,0.078309,0.055404
에스엘바이오닉스,0.097334,-18.374621,-0.586949,-0.578525,0.639343,-0.056904,0.133448,0.502452,-0.276396,0.455392,0.038316,-0.110476,0.011180
나노브릭,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
아세아텍,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
